In [1]:
from lib.project_5 import load_data_from_database, make_data_dict, general_model, general_transformer

# Step 1 - Benchmarking

In [2]:
data = load_data_from_database()

In [3]:
print data.shape
data.head()

(2000, 502)


,index,feat_000,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,...,feat_491,feat_492,feat_493,feat_494,feat_495,feat_496,feat_497,feat_498,feat_499,label
0,0,485,477,537,479,452,471,491,476,475,...,481,477,485,511,485,481,479,475,496,-1
1,1,483,458,460,487,587,475,526,479,485,...,478,487,338,513,486,483,492,510,517,-1
2,2,487,542,499,468,448,471,442,478,480,...,481,492,650,506,501,480,489,499,498,-1
3,3,480,491,510,485,495,472,417,474,502,...,480,474,572,454,469,475,482,494,461,1
4,4,484,502,528,489,466,481,402,478,487,...,479,452,435,486,508,481,504,495,511,1


In [4]:
data_dict = make_data_dict(data)

In [7]:
data_dict['X_train'].head()

,feat_000,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_490,feat_491,feat_492,feat_493,feat_494,feat_495,feat_496,feat_497,feat_498,feat_499
1412,478,499,535,477,474,469,460,476,474,481,...,484,480,532,403,496,524,480,490,476,509
1946,473,483,443,488,405,485,491,477,520,494,...,489,476,516,519,506,485,479,481,455,488
1584,500,510,457,491,475,477,483,476,483,476,...,483,471,515,598,509,453,470,486,550,472
1908,471,537,436,489,521,479,529,476,498,480,...,459,480,478,714,538,516,483,488,508,558
314,479,506,499,483,557,462,480,477,519,484,...,482,479,500,506,518,553,476,497,501,502


**NOTE: EACH OF THESE SHOULD BE WRITTEN SOLELY WITH REGARD TO STEP 1 - BENCHMARKING**

### Domain and Data

**TODO:** Write a simple statement about the domain of your problem and the dataset upon which you will be working. 

### Problem Statement

**TODO:** Write a simple problem statement with regard to benchmarking your work only.

### Solution Statement

**TODO:** Write a simple solution statement with regard to benchmarking your work only.

### Metric

**TODO**: Write a statement about the metric you will be using. This section is global as it will be the metric you will use throughout this project. Provide a brief justification for choosing this metric.

### Benchmark

**TODO**: Write a statement explaining that this is the process by which you identify a benchmark for your project.

## Implementation

Implement the following code pipeline using the functions you write in `lib/project_5.py`.

<img src="assets/benchmarking.png" width="600px">